In [92]:
# importing packages
from datetime import datetime
start_time = datetime.now()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)

**selecting necessary data**

In [25]:
#top 30 variables

topvars = pd.read_csv('top_30_vars_final.csv')
topvars = list(topvars.var_name.values)

#all data
data = pd.read_csv('card_transactions_all_features.csv')

In [26]:
columnskeep = topvars + ['Recnum','Date','Fraud']
len(columnskeep)

33

In [27]:
df = data[columnskeep]

In [28]:
df.head()

,Cardnum_max_30,card_merchant_max_30,Merchnum_total_3,Cardnum_total_130,Merchnum_mean_7,card_merchant_median_1,Merchnum_mean_3,Cardnum_mean_30,card_merchant_mean_1,card_state_mean_30,...,card_zip_max_30,Merchnum_mean_1,card_merchant_mean_3,card_merchant_max_14,card_merchant_max_7,card_merchant_mean_7,card_zip_max_7,Recnum,Date,Fraud
0,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,2010-01-01,0
1,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2,2010-01-01,0
2,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3,2010-01-01,0
3,0.00,0.00,3.62,NaN,3.62,0.00,3.62,0.00,0.00,0.00,...,0.00,3.62,0.00,0.00,0.00,0.00,0.00,4,2010-01-01,0
4,3.62,3.62,7.24,30.0,3.62,3.62,3.62,3.62,3.62,3.62,...,3.62,3.62,3.62,3.62,3.62,3.62,3.62,5,2010-01-01,0


In [29]:
df = df.fillna(0)

**splitting modeling and oot data**

In [32]:
main_Xs = topvars

#oot df
df_oot = df[df.Date>='2010-11-01'].copy()

#x and y oot
y_oot = df_oot['Fraud'].copy()
X_oot = df_oot.copy().drop(columns='Fraud')

#modeling data
df_model = df[(df.Date>'2010-01-14')&(df.Date<'2010-11-01')].copy()

# #x and y train
y = df_model['Fraud'].copy()
X = df_model.copy().drop(columns='Fraud')

# #dropping date and record
df_oot.drop(columns=['Date','Recnum'], inplace=True)
y_oot.drop(columns=['Date','Recnum'], inplace=True)
X_oot.drop(columns=['Date','Recnum'], inplace=True)
df_model.drop(columns=['Date','Recnum'], inplace=True)
y.drop(columns=['Date','Recnum'], inplace=True)
X.drop(columns=['Date','Recnum'], inplace=True)

# X_oot = StandardScaler().fit_transform(X_oot)
# X = StandardScaler().fit_transform(X)

In [33]:
X

,Cardnum_max_30,card_merchant_max_30,Merchnum_total_3,Cardnum_total_130,Merchnum_mean_7,card_merchant_median_1,Merchnum_mean_3,Cardnum_mean_30,card_merchant_mean_1,card_state_mean_30,...,card_zip_mean_7,card_state_mean_7,card_state_median_7,card_zip_max_30,Merchnum_mean_1,card_merchant_mean_3,card_merchant_max_14,card_merchant_max_7,card_merchant_mean_7,card_zip_max_7
3338,1593.38,983.66,520.59,4.530289,291.155000,0.000,130.147500,403.136500,0.000,813.400000,...,813.400000,813.400000,813.40,983.66,0.000000,0.000000,983.66,983.66,813.400000,983.66
3339,656.50,15.40,827.47,16.660322,6.272880,0.000,7.522455,10.018220,0.000,4.492735,...,4.812286,4.812286,3.80,15.40,91.000000,5.194839,15.40,15.40,4.706765,15.40
3340,2472.00,554.38,311.01,5.433914,155.505000,155.505,155.505000,454.118438,155.505,267.456667,...,155.505000,227.690000,278.73,554.38,155.505000,155.505000,554.38,278.73,155.505000,278.73
3341,656.50,15.40,831.84,16.709453,6.262969,4.370,7.494054,9.970756,4.370,4.491695,...,4.806056,4.806056,3.80,15.40,47.685000,5.169062,15.40,15.40,4.701884,15.40
3342,0.00,0.00,113.87,0.000000,241.670000,0.000,113.870000,0.000000,0.000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83965,929.00,0.00,606.55,0.000000,288.095000,0.000,151.637500,355.532500,0.000,0.000000,...,0.000000,0.000000,0.00,0.00,419.120000,0.000000,0.00,0.00,0.000000,0.00
83966,179.98,179.98,595.77,30.000000,119.154000,179.980,119.154000,179.980000,179.980,179.980000,...,179.980000,179.980000,179.98,179.98,159.143333,179.980000,179.98,179.98,179.980000,179.98
83967,399.15,0.00,0.00,0.000000,0.000000,0.000,0.000000,217.116667,0.000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00
83968,0.00,0.00,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00


In [34]:
#caclulate fdr at 3%

def calculateFDR(predict, fraudscore, y):
    temp = pd.DataFrame({'true':y.tolist(), 'predict':predict, 'score':fraudscore})
    temp.sort_values('score', ascending=False, inplace=True)
    percent3 = int(len(predict)*0.03)
    return np.sum(temp.true[0:percent3])/np.sum(temp.true)

**Boosted Trees**

In [59]:
def boostedtreeFDR(X, X_oot, y, y_oot, n_estimators, learning_rate, max_depth, iteration=10):
    bt = GradientBoostingClassifier(n_estimators = n_estimators, \
                                    learning_rate = learning_rate, \
                                    max_depth = max_depth, \
                                    random_state=0)
    
    fdr_df=[]
    fdr_total=[0,0,0] # train, test, oot
    for i in range(iteration):
        X_train, X_test, y_train, y_test = train_test_split(X, y) # using default setting for test_size(0.25), random_state(None) and shuffle(True)
        bt.fit(X_train, y_train)
        
        data = [[X_train, y_train], [X_test, y_test], [X_oot, y_oot]]
        fdr_list=[]
        for d in range(len(data)):
            predict = bt.predict(data[d][0])
            prob = bt.predict_proba(data[d][0])
            fraudscore = prob.transpose()[1]
            fdr_each = calculateFDR(predict, fraudscore, data[d][1])
            fdr_total[d] += fdr_each
            fdr_list.append(fdr_each)
        
        fdr_df.append(fdr_list)
    
    avg_train = fdr_total[0] / iteration
    avg_test = fdr_total[1] / iteration
    avg_oot = fdr_total[2] / iteration
    result = [avg_train, avg_test, avg_oot]

    return fdr_df, result

In [168]:
### apply function here
bt_start_time = datetime.now()
print(bt_start_time)
# Adjusting hyperparameters
n_estimators = [800, 1000, 1200] # original: [100, 200, 300]
learning_rate = [0.1, 0.05] # original: [0.1]
depth = [2, 3, 4] # original: [3, 4, 5]

maximum = 0
params=[]
bt_fdr_all=[]
bt_fdr_mean=[]
for i in n_estimators:
    for j in learning_rate:
        for k in depth:
            temp1, temp2 = boostedtreeFDR(X, X_oot, y, y_oot, iteration=10, \
                                          n_estimators = i, \
                                          learning_rate = j, \
                                          max_depth = k) # optimizing on test data
            params.append([i, j, k])
            bt_fdr_mean.append(temp2)
            bt_fdr_all.append(temp1)
            if maximum < temp2[1]:
                maximum=temp2[1]
                maxfdr=temp2
                maxparams=[i,j,k]
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print(f'Done -------------\t{[i,j,k]}\twith test fdr: {temp2[1]:.4f}\t{current_time}\
            \nCurrent maxparams:\t{maxparams}\twith test fdr: {maximum:.4f}\n')

bt_end_time = datetime.now()
print(f'n_estimators:\t\t{maxparams[0]}\n\
learning_rate:\t\t{maxparams[1]}\n\
max_depth:\t\t{maxparams[2]}\n\
with average test fdr:\t{maximum:.4f}')

# Duration of the cell
print('Duration: ', bt_end_time - bt_start_time)

2021-04-28 13:02:55.494286
Done -------------	[800, 0.1, 2]	with test fdr: 0.8713	            2021-04-28 13:22:39
Current maxparams:	[800, 0.1, 2]             	with test fdr: 0.8713

Done -------------	[800, 0.1, 3]	with test fdr: 0.8706	            2021-04-28 13:51:11
Current maxparams:	[800, 0.1, 2]             	with test fdr: 0.8713

Done -------------	[800, 0.1, 4]	with test fdr: 0.8737	            2021-04-28 14:29:30
Current maxparams:	[800, 0.1, 4]             	with test fdr: 0.8737

Done -------------	[800, 0.05, 2]	with test fdr: 0.8441	            2021-04-28 14:49:11
Current maxparams:	[800, 0.1, 4]             	with test fdr: 0.8737

Done -------------	[800, 0.05, 3]	with test fdr: 0.8865	            2021-04-28 15:18:21
Current maxparams:	[800, 0.05, 3]             	with test fdr: 0.8865

Done -------------	[800, 0.05, 4]	with test fdr: 0.8791	            2021-04-28 15:56:40
Current maxparams:	[800, 0.05, 3]             	with test fdr: 0.8865

Done -------------	[1000, 0.1, 2

In [169]:
print(f'n_estimators:\t\t{maxparams[0]}\n\
learning_rate:\t\t{maxparams[1]}\n\
max_depth:\t\t{maxparams[2]}\n\
with average test fdr:\t{maximum:.4f}')

n_estimators:		1400
learning_rate:		0.05
max_depth:		3
with average test fdr:	0.8946


In [170]:
bt_output30 = pd.DataFrame({'params':params, 'fdr':bt_fdr_mean})
bt_output30_alliter = pd.DataFrame(bt_fdr_all[0],columns=[str(params[0])+': train', 'test', 'oot'])
for i in range(1,len(bt_fdr_all)):
    bt_output30_alliter = pd.concat([bt_output30_alliter, pd.DataFrame(bt_fdr_all[i],columns=[str(params[i])+': train', 'test', 'oot'])], axis=1)

In [171]:
bt_output30

,params,fdr
0,"[800, 0.1, 2]","[0.9510681995030049, 0.8713185712254813, 0.386..."
1,"[800, 0.1, 3]","[0.9895550727182941, 0.8706469218246944, 0.362..."
2,"[800, 0.1, 4]","[0.9990817271188523, 0.8737093633614993, 0.334..."
3,"[800, 0.05, 2]","[0.9088703433330568, 0.8441181985843041, 0.409..."
4,"[800, 0.05, 3]","[0.9623488743123515, 0.8864675895964, 0.384357..."
5,"[800, 0.05, 4]","[0.9893643713826006, 0.879116966938728, 0.3921..."
6,"[1000, 0.1, 2]","[0.9619822929014601, 0.8678945496280791, 0.345..."
7,"[1000, 0.1, 3]","[0.9910268820124862, 0.8682503234997581, 0.377..."
8,"[1000, 0.1, 4]","[0.9992319049527824, 0.8465494339582813, 0.286..."
9,"[1000, 0.05, 2]","[0.9166312798096194, 0.8700213627947786, 0.389..."


In [172]:
bt_output30_alliter

,"[800, 0.1, 2]: train",test,oot,"[800, 0.1, 3]: train",test,oot,"[800, 0.1, 4]: train",test,oot,"[800, 0.05, 2]: train",test,oot,"[800, 0.05, 3]: train",test,oot,"[800, 0.05, 4]: train",test,oot,"[1000, 0.1, 2]: train",test,oot,"[1000, 0.1, 3]: train",test,oot,"[1000, 0.1, 4]: train",test,oot,"[1000, 0.05, 2]: train",test,oot,"[1000, 0.05, 3]: train",test,oot,"[1000, 0.05, 4]: train",test,oot,"[1200, 0.1, 2]: train",test,oot,"[1200, 0.1, 3]: train",test,oot,"[1200, 0.1, 4]: train",test,oot,"[1200, 0.05, 2]: train",test,oot,"[1200, 0.05, 3]: train",test,oot,"[1200, 0.05, 4]: train",test,oot,"[1400, 0.1, 2]: train",test,oot,"[1400, 0.1, 3]: train",test,oot,"[1400, 0.1, 4]: train",test,oot,"[1400, 0.05, 2]: train",test,oot,"[1400, 0.05, 3]: train",test,oot,"[1400, 0.05, 4]: train",test,oot
0,0.952599,0.892523,0.324022,0.978788,0.841346,0.312849,0.996983,0.892683,0.418994,0.908507,0.857143,0.396648,0.966817,0.897561,0.368715,0.993902,0.816038,0.424581,0.954407,0.914286,0.312849,0.995363,0.859729,0.402235,1.000000,0.801887,0.268156,0.922610,0.827751,0.379888,0.975232,0.887387,0.463687,0.995363,0.850679,0.324022,0.965079,0.890756,0.407821,0.997019,0.878173,0.268156,1.000000,0.835000,0.268156,0.927357,0.850679,0.374302,0.981538,0.862385,0.513966,0.996885,0.867257,0.357542,0.974046,0.873239,0.418994,0.995215,0.842324,0.212291,1.000000,0.834123,0.245810,0.941896,0.869159,0.391061,0.993912,0.890995,0.391061,0.998433,0.860870,0.351955
1,0.949287,0.886076,0.391061,0.990937,0.893204,0.474860,0.998447,0.879464,0.329609,0.899248,0.817734,0.413408,0.961890,0.882075,0.374302,0.990726,0.914027,0.268156,0.960692,0.853448,0.374302,0.993865,0.870370,0.340782,1.000000,0.858407,0.307263,0.919818,0.855072,0.407821,0.963581,0.880383,0.279330,0.996890,0.897778,0.413408,0.968992,0.883408,0.374302,0.993958,0.873786,0.324022,0.998476,0.858491,0.262570,0.928680,0.866029,0.396648,0.985959,0.889868,0.335196,0.995420,0.892019,0.324022,0.972477,0.869159,0.363128,0.998366,0.859375,0.301676,1.000000,0.870536,0.279330,0.936826,0.872146,0.357542,0.987520,0.850220,0.452514,0.998503,0.875000,0.379888
2,0.948092,0.910798,0.329609,0.980031,0.894009,0.351955,0.998466,0.861111,0.256983,0.894100,0.874396,0.379888,0.958991,0.893162,0.335196,0.977953,0.871245,0.446927,0.954887,0.881773,0.324022,0.986154,0.857798,0.357542,0.995370,0.863636,0.346369,0.903323,0.907767,0.413408,0.984424,0.849558,0.474860,0.993730,0.908696,0.474860,0.968603,0.896104,0.413408,0.996830,0.886076,0.340782,0.998435,0.812227,0.346369,0.920732,0.877358,0.363128,0.981073,0.854701,0.346369,0.998433,0.869565,0.268156,0.974281,0.908213,0.351955,0.998452,0.900901,0.301676,0.998435,0.847162,0.385475,0.926448,0.899563,0.402235,0.987261,0.908333,0.385475,0.998469,0.879070,0.363128
3,0.955316,0.840183,0.430168,0.993874,0.846512,0.307263,1.000000,0.858447,0.374302,0.901690,0.866359,0.391061,0.963691,0.898551,0.312849,0.989645,0.895833,0.452514,0.963020,0.858447,0.290503,0.993769,0.893805,0.374302,0.998462,0.830275,0.340782,0.904468,0.890411,0.402235,0.976672,0.857778,0.385475,0.996914,0.840909,0.379888,0.965774,0.903061,0.391061,1.000000,0.866667,0.318436,0.998553,0.875706,0.318436,0.936460,0.879227,0.385475,0.980333,0.864734,0.396648,0.996890,0.893333,0.418994,0.979969,0.876712,0.418994,0.998469,0.827907,0.251397,1.000000,0.846890,0.296089,0.950693,0.872146,0.430168,0.988924,0.915254,0.430168,0.998501,0.895522,0.318436
4,0.948195,0.865801,0.452514,0.990977,0.847291,0.407821,1.000000,0.852814,0.441341,0.910294,0.872340,0.396648,0.959641,0.884422,0.379888,0.989011,0.874459,0.486034,0.973561,0.800000,0.296089,0.991843,0.854902,0.469274,1.000000,0.817259,0.268156,0.924051,0.864407,0.418994,0.980983,0.881857,0.312849,0.993930,0.875598,0.413408,0.979814,0.857143,0.379888,0.995238,0.865546,0.452514,0.995595,0.860963,0.418994,0.942006,0.834783,0.312849,0.987805,0.886792,0.357542,0.996942,0.911215,0.463687,0.972093,0.878924,0.324022,0.993958,0.878641,0.391061,1.000000,0.865741,0.357542,0.937008,0.866953,0.441341,0.985

In [173]:
bt_output30.to_csv('results/bt_30vars_mean.csv')
bt_output30_alliter.to_csv('results/bt_30vars_all.csv')

In [174]:
# Creating Summary Table
bt_summary = bt_output30.copy()
bt_summary['Number of Features'] = 30
bt_summary['# of Estimators'] = bt_summary['params'].apply(lambda x: x[0])
bt_summary['Learning Rate'] = bt_summary['params'].apply(lambda x: x[1])
bt_summary['Max Depth'] = bt_summary['params'].apply(lambda x: x[2])

bt_summary['Train'] = bt_summary['fdr'].apply(lambda x: x[0])
bt_summary['Test'] = bt_summary['fdr'].apply(lambda x: x[1])
bt_summary['OOT'] = bt_summary['fdr'].apply(lambda x: x[2])
bt_summary.drop(columns = ['params', 'fdr'], inplace = True)
bt_summary

,Number of Features,# of Estimators,Learning Rate,Max Depth,Train,Test,OOT
0,30,800,0.10,2,0.951068,0.871319,0.386034
1,30,800,0.10,3,0.989555,0.870647,0.362570
2,30,800,0.10,4,0.999082,0.873709,0.334078
3,30,800,0.05,2,0.908870,0.844118,0.409497
4,30,800,0.05,3,0.962349,0.886468,0.384358
5,30,800,0.05,4,0.989364,0.879117,0.392179
6,30,1000,0.10,2,0.961982,0.867895,0.345810
7,30,1000,0.10,3,0.991027,0.868250,0.377654
8,30,1000,0.10,4,0.999232,0.846549,0.286034
9,30,1000,0.05,2,0.916631,0.870021,0.389944


In [175]:
bt_summary.to_excel('Results/bt_summary.xlsx', index = False)